In [1]:
import numpy as np

In [2]:
def haversine(lon1, lat1, lon2, lat2):
    r = 6371  

    lon1_rad = np.radians(lon1)
    lat1_rad = np.radians(lat1)
    lon2_rad = np.radians(lon2)
    lat2_rad = np.radians(lat2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = r * c

    return distance

In [3]:
def t_advection(mlt, mlu, mlv, lon, lat):
    # Description - advection term of heat budget
    # input
    # mlt - 3D daily mixed-layer temperature data to calculate advection term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # mlu - 3D daily mixed-layer zonal velocity data to calculate advection term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # mlv - 3D daily mixed-layer meridional velocity data to calculate advection term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # mlt & mlu & mlv must have the same dimensions.
    # lon - A vector, longitude range of temperatures and horizontal velocity
    # lat - A vector, latitude range of temperatures and horizontal velocity
    # output
    # adv - horizontal advection term of heat budget
    # advu - zonal advection term of heat budget
    # advv - meridional advection term of budget

    nx, ny, nt = mlt.shape

    # Calculate grid spacing
    dx = np.zeros((nx, ny, nt))
    for j in range(len(lat)):
        for i in range(1, len(lon) - 1):
            dx[i, j, :] = haversine(lon[i-1], lat[j], lon[i+1], lat[j]) * 1e3
    dx[0, :, :] = dx[1, :, :]
    dx[-1, :, :] = dx[-2, :, :]
    dy = haversine(lon[0], lat[0], lon[0], lat[1]) * 1000 * 2

    # Calculate advection terms
    advu0 = np.full((nx, ny, nt), np.nan)  # unit: °C/day
    advv0 = np.full((nx, ny, nt), np.nan)  # unit: °C/day
    advu0[1:-1, 1:-1, :] = -(mlu[1:-1, 1:-1, :] * (mlt[2:, 1:-1, :] - mlt[:-2, 1:-1, :]) / dx[1:-1, 1:-1, :]) * 3600 * 24
    advv0[1:-1, 1:-1, :] = -(mlv[1:-1, 1:-1, :] * (mlt[1:-1, 2:, :] - mlt[1:-1, :-2, :]) / dy) * 3600 * 24

    advu = (advu0[:, :, :-1] + advu0[:, :, 1:]) / 2
    advv = (advv0[:, :, :-1] + advv0[:, :, 1:]) / 2
    adv = advu + advv

    return adv, advu, advv

In [30]:
def t_surfaceforcing(Qnet, Qdsw, mld, *args):
    # Description - surface forcing term of heat budget
    # input
    # Qnet - 3D daily net air-sea heat flux data to calculate surface forcing term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # Qdsw - 3D daily downward shortwave radiation data to calculate surface forcing term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # mld - 3D daily mixed-layer depth data to calculate surface forcing term of heat budget, specified as a
    #   three dimensions matrix m-by-n-by-t. m and n separately indicate two spatial dimensions
    #   and t indicates temporal dimension.
    # Q & Qdsw & mld must have the same dimensions.
    # rho - seawater density, default is 1025 kg/m3.
    # Cp - seawater heat capacity, default is 3986 J/(kg°C).
    # r, c1, c2 - defaults are 0.58, 0.35, 23, respectively. (Paulson and Simpon, 1977)
    # output
    # sf - surface forcing term of heat budget

    paramNames = ['rho', 'Cp', 'r', 'c1', 'c2']
    defaults = [1025, 3986, 0.58, 0.35, 23]

    # Parsing optional arguments
    rho, Cp, r, c1, c2 = defaults[:len(args)] if len(args) > 0 else defaults

    Qd = Qdsw * (r * np.exp(-mld / c1) + (1 - r) * np.exp(
        -mld / c2))  # shortwave radiation penetrating below the mixed layer
    sf0 = (Qnet - Qd) / rho / Cp / mld * 3600 * 24  # unit: °C/day
    sf = (sf0[:, :, :-1] + sf0[:, :, 1:]) / 2

    return sf

In [31]:
m, n, t = 10, 10, 365
Qnet = np.random.rand(m, n, t)
Qdsw = np.random.rand(m, n, t)
mld = np.random.rand(m, n, t)

sf = t_surfaceforcing(Qnet, Qdsw, mld)

print(sf.shape) 

(10, 10, 364)


array([[[-2.89551941e-02,  2.96337313e-02,  3.60889932e-02, ...,
          6.85302708e-03,  4.02845885e-03,  2.27345646e-03],
        [-3.68892702e-03,  5.26586664e-02,  5.16650644e-02, ...,
          9.81307337e-03,  3.51370887e-03,  5.29718352e-03],
        [-1.93319948e-02, -1.03529094e-03,  3.53307556e-02, ...,
         -8.26069673e-03, -4.40323188e-03,  1.12796086e-02],
        ...,
        [ 1.25257615e-02,  1.04673517e-02, -2.94786353e-03, ...,
          2.15519740e-03,  1.48223233e-02,  1.16714024e-02],
        [ 2.29749282e-02, -9.01500839e-04, -1.03470040e-02, ...,
          1.33104311e-02,  1.50130284e-02,  1.02479851e-02],
        [ 2.48848207e-03, -2.67072958e-04,  1.49560383e-01, ...,
         -1.60490876e-03,  1.71356251e-02,  1.25456728e-02]],

       [[-4.43271077e-02, -3.15095778e-02,  1.47181303e-02, ...,
          1.62409885e-02,  1.68161048e-02,  1.60444664e-02],
        [-1.97430053e-02,  4.48161427e-02,  7.39309518e-02, ...,
          1.04416885e-02,  1.94904103e